# Home Credit Default Risk - EDA POS_CASH_balance

## 1. Introduction

**Context:**

This notebook contains basic EDA for POS CASH balance data set.

This is additional source of data (application_train/application_test are the main training and testing data).

POS_CASH_balance.csv

    Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
    This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

**Goals:**

    To comprehensively understand the dataset's structure, identify key patterns, and discover meaningful insights that will inform a robust feature engineering and modeling strategy.

**Objectives:**

    Conduct a comprehensive Exploratory Data Analysis (EDA): Perform an in-depth exploration of the datasets to understand their statistical properties and distributions.

    Identify and address data quality issues: Investigate missing values, identify and handle data anomalies.

    Analyze feature relationships: Evaluate correlations between features and assess their individual relationships with the target variable to prioritize their importance for the model.

    Leverage automated tools for initial insights: Utilize libraries like Sweetviz to quickly generate an initial feature exploration report.


## 2. Exploratory Data Analysis (EDA)

### A. Data loading & Initial checks

In [1]:
%load_ext jupyter_black

In [2]:
import pandas as pd
import numpy as np
import sys
import os
from typing import Dict, Optional, List, Tuple, Union
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sweetviz.graph")
import sweetviz as sv
from ydata_profiling import ProfileReport
from IPython.display import IFrame

In [3]:
sys.path.append(os.path.abspath(".."))
from Data.utils_EDA import feature_types, missing_columns, calculate_missing_rows
from Data.utils_modeling import downcast_numeric_col

In [4]:
pos = pd.read_csv(r"..\Data\POS_CASH_balance.csv")
pos.shape

(10001358, 8)

**Downcasting numeric columns**

In [5]:
pos = pos.copy()
downcast_numeric_col(pos)
pos.dtypes.unique()

array([dtype('int32'), dtype('int8'), dtype('float32'), dtype('O'),
       dtype('int16')], dtype=object)

In [6]:
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [7]:
pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int32  
 1   SK_ID_CURR             int32  
 2   MONTHS_BALANCE         int8   
 3   CNT_INSTALMENT         float32
 4   CNT_INSTALMENT_FUTURE  float32
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int16  
 7   SK_DPD_DEF             int16  
dtypes: float32(2), int16(2), int32(2), int8(1), object(1)
memory usage: 276.6+ MB


**Feature descriptions:**


1. SK_ID_PREV ,"ID of previous credit in Home Credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)",

2. SK_ID_CURR,ID of loan in our sample,

3. MONTHS_BALANCE,"Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )",time only relative to the application

4. CNT_INSTALMENT,Term of previous credit (can change over time),

5. CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,

6. 6 .NAME_CONTRACT_STATUS,Contract status during the month,

7. SK_DPD,DPD (days past due) during the month of previous credit,

8. SK_DPD_DEF,DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

**Feature types**

In [8]:
feature_types(pos)

Numerical features: ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']
Categorical features: ['NAME_CONTRACT_STATUS']
Binary features: []


In [9]:
pos.dtypes.value_counts()

int32      2
float32    2
int16      2
int8       1
object     1
Name: count, dtype: int64

In [10]:
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

pos.describe().T

,count,mean,std,min,25%,50%,75%,max
SK_ID_PREV,10001358.0,1.903217e+06,535846.530722,1000001.0,1434405.0,1896565.0,2368963.0,2843499.0
SK_ID_CURR,10001358.0,2.784039e+05,102763.745090,100001.0,189550.0,278654.0,367429.0,456255.0
MONTHS_BALANCE,10001358.0,-3.501259e+01,26.066570,-96.0,-54.0,-28.0,-13.0,-1.0
CNT_INSTALMENT,9975287.0,1.708964e+01,11.995056,1.0,10.0,12.0,24.0,92.0
CNT_INSTALMENT_FUTURE,9975271.0,1.048384e+01,11.109057,0.0,3.0,7.0,14.0,85.0
SK_DPD,10001358.0,1.160693e+01,132.714043,0.0,0.0,0.0,0.0,4231.0
SK_DPD_DEF,10001358.0,6.544684e-01,32.762491,0.0,0.0,0.0,0.0,3595.0


**Key insights:**

CNT_INSTALMENT, CNT_INSTALMENT_FUTURE: Most loans are short- to medium-term, with relatively few future installments remaining. 

SK_DPD, SK_DPD_DEF: The majority of borrowers are not delinquent, but there are significant outliers with extreme delays and defaults.

**Missing values**

In [11]:
missing_columns(pos)

,Missing Count,Missing Count Ratio,Missing Count %
CNT_INSTALMENT_FUTURE,26087,0.002608,0.3
CNT_INSTALMENT,26071,0.002607,0.3


In [12]:
calculate_missing_rows(pos)

Missing rows: 26184 of 10001358 total rows in data set.
Missing rows %: 0.26


Almost no missing values. We will use imputation.

**Checking for duplicates.**

In [13]:
print(
    f"Duplicates: {pos.duplicated().sum()}, {(pos.duplicated().sum() / len(pos) * 100):.2f}%"
)

Duplicates: 0, 0.00%


No duplicates in this dataset.

**Sweetviz report**

We can find the report in EDA folder.

In [ ]:
report = sv.analyze(df)
html_file = f"POS_CASH_balance_sweetviz_report.html"
report.show_html(html_file)
#display(IFrame(html_file, width=950, height=600))

**Ydata Report**

We can find the report in EDA folder.

In [ ]:
profile = ProfileReport(df, title="POS_CASH_balance EDA", explorative=True)

profile.to_file("POS_CASH_balance_EDA.html")

### B. Feature analysis

    MONTHS_BALANCE - Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly ),time only relative to the application

Numerical, no missing values, no zeros.

Minimum	-96 (8 years), Maximum	-1, Mean -35.0. Left skewed. Outliers.

    CNT_INSTALMENT - Term of previous credit (can change over time)

High correlation with CNT_INSTALMENT_FUTURE.

Numerical, 0.3% missing values, no zeros.

Minimum	1, Maximum 92 (7.7 years), Mean	17.1. Right skewed. Outliers.

    CNT_INSTALMENT_FUTURE - Installments left to pay on the previous credit

High correlation with CNT_INSTALMENT.

Numerical, 0.3% missing values, 11.9% zeros.

Minimum	0, Maximum	85 (7.1 years), Mean 10.5. Right skewed. Outliers.

    Feature engineering:
    - how much credit left to pay INSTALMENTS_COMPLETED_RATIO = 1 - CNT_INSTALMENT_FUTURE / (CNT_INSTALMENT +1)

    NAME_CONTRACT_STATUS - Contract status during the month

Categorical, no missing values, imbalanced: Active 91.5%, Completed 7.4%.

    Flags for Active, Completed and Other (Signed, Demand, Returned to the store, Approved, Amortized debt, Canceled).

    SK_DPD - DPD (days past due) during the month of previous credit

Numerical, no missing values, 97.0% zeros.

Minimum	0, Maximum 4,231 (~11.6 years), Mean 11.6. Outliers.

    Clip outliers on 365 (beyond 1 year, risk is already extreme).
    Feature engineering:
    - Delinquency flags:
        IS_DELINQUENT = SK_DPD > 0
        IS_SERIOUSLY_DELINQUENT = SK_DPD > 30
    - Calculate difference between last and first SK_DPD for deterioration detection

    SK_DPD_DEF - DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

Numerical, no missing values, 98.9% zeros.

Minimum	0, Maximum 3595 (~9.85 years), Mean 0.65. Outliers.

    Clip outliers on 365 (beyond 1 year, risk is already extreme).
    Feature engineering:
    - Delinquency flag SK_DPD_DEF > 0

### Correlation

We will analyze the relationships between features using a Ydata-Quality report. This report will provide a comprehensive overview of our data, including an automated correlation matrix for all features.

To determine which features are most impactful for our model, we will use a more robust method: LightGBM's feature importance. After aggregating the columns from specific datasets into our main dataset, the LightGBM model will automatically calculate the importance of each feature in predicting the target variable. This approach is superior as it directly assesses a feature's predictive power within the context of our chosen model, providing a more reliable measure of its relationship with the target.

**Feature Relationships**

High correlation (Ydata Report):
    CNT_INSTALMENT - CNT_INSTALMENT_FUTURE

## 3. Summary

**Key EDA findings for POS CASH balance:**

    - Total features: 8 (numeric 7, categorical 1), rows: ~ 10.0M,

    - Missing cells	<0.1%, rows with missing values - 0.26%,
    
    - Missing values (>15%): none
        
    - Negative values (>50%):
        - MONTHS_BALANCE - 100.0%

    - Zeros (>30%):
        - SK_DPD - 97.0%
        - SK_DPD_DEF - 98.9%

    - Strong correlations (>0.7):
        CNT_INSTALMENT - CNT_INSTALMENT_FUTURE
    
    - Duplicates: none

**Planned Feature Engineering: POS/CASH Balance**

Ideas for feature engineering from the POS/CASH balance dataset to capture a client’s repayment behavior, delinquency risk, and contract performance. The main steps:

    1. Handling Raw DPD Values

        - Clipp extreme delinquency values (SK_DPD and SK_DPD_DEF) at 365 days to reduce the impact of outliers.

    2. Delinquency Behavior Features

        - Flags to capture whether the client has ever been late:

            IS_DELINQUENT - any late payment (> 0 days)

            IS_SERIOUSLY_DELINQUENT - severe delinquency (> 30 days)

            IS_DELINQUENT_DEF - default delinquency flag from data

        - Aggregations for DPD metrics:

            SK_DPD_max - worst-ever delinquency
            
            SK_DPD_mean - average delinquency
            
            SK_DPD_last - most recent delinquency
            
            SK_DPD_std - volatility in delinquency

        - Trend feature:

            DPD_CHANGE - difference between last and first DPD (improving or deteriorating)

    3. Contract status indicators

        - Create binary flags for contract states:

            IS_ACTIVE - proportion of months with active status
            
            IS_COMPLETED - ever successfully completed
            
            IS_OTHER_STATUS - any unusual status (e.g., Signed, Demand, Returned)

    4. Repayment Progress

        - INSTALMENTS_COMPLETED_RATIO - ratio of completed installments based on CNT_INSTALMENT and CNT_INSTALMENT_FUTURE

            Aggregated as mean and max to capture overall repayment progress

    5. Credit History Length

        From original columns:

            CNT_INSTALMENT - average and maximum term length
            
            CNT_INSTALMENT_FUTURE - remaining installments (mean, min, max)
            
            MONTHS_BALANCE - min, max, and mean (duration of POS history)

    6. Aggregation Strategy

        - Group by SK_ID_CURR to summarize all POS/CASH history per client

        - Combine custom engineered features and standard numeric aggregations

        - Select most predictive features using LightGBM importance + ROC-AUC ranking

        - Must-keep features:

            POS_SK_DPD_max - Maximum number of days past due across POS cash loans.
            
            POS_IS_DELINQUENT_max – Flag if the client was ever delinquent on POS loans.
            
            POS_IS_SERIOUSLY_DELINQUENT_max – Flag if the client was ever seriously delinquent.
            
            POS_SK_DPD_DEF_max – Maximum number of days past due with default-level severity.
            
            POS_INSTALMENTS_COMPLETED_RATIO_mean - Average ratio of completed installments to total installments.
            
            POS_IS_COMPLETED_max – Flag if any POS loan was fully completed.

    7. Feature Selection

        Use LightGBM importance + ROC-AUC ranking to select top features.

        Merge selected features back to main data frame for model training.